<a href="https://colab.research.google.com/github/maro525/ml_ColabNotebooks/blob/master/dcgan_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pwd
!ls

/content
datalab  discriminator	drive  generator


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# create auth token
from google.colab import auth
auth.authenticate_user()

gpg: keybox '/tmp/tmpjean_ev4/pubring.gpg' created
gpg: /tmp/tmpjean_ev4/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [0]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

KeyboardInterrupt: ignored

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pwd
!ls /content/drive/Notebooks
import os
os.chdir('/content/drive/Notebooks/gan')

/content/drive/Notebooks/gan
gan  segment  Soundscape  world_image


In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
import numpy as np
from PIL import Image
import math

Using TensorFlow backend.


In [0]:
def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, output_dim=1024))
    model.add(Activation('tanh'))
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(64, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    return model

In [0]:
def discriminator_model():
    model = Sequential()
    model.add(
            Conv2D(64, (5, 5),
            padding='same',
            input_shape=(28, 28, 1))
            )
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [0]:
def generator_containing_discriminator(g, d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)
    return model

In [0]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                      dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, 0]
    return image

In [0]:
def train(BATCH_SIZE):
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    # 正規化
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train[:, :, :, None]
    X_test = X_test[:, :, :, None]
    d = discriminator_model()
    g = generator_model()
    d_on_g = generator_containing_discriminator(g, d)
    d_optim = SGD(lr=0.005, momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.005, momentum=0.9, nesterov=True)
    g.compile(loss='binary_crossentropy', optimizer=g_optim)
    d_on_g.compile(loss='binary_crossentropy', optimizer=g_optim)
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    for epoch in range(100):
      print("Epoch is", epoch)
      print("Number of baches", int(X_train.shape[0]/BATCH_SIZE))
      for index in range(int(X_train.shape[0]/BATCH_SIZE)):
          noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
          image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
          generated_images = g.predict(noise, verbose=0)
          if index % 20 == 0:
              image = combine_images(generated_images)
              image = image*127.5+127.5
              Image.fromarray(image.astype(np.uint8)).save(
                  str(epoch)+"_"+str(index)+".png")
          X = np.concatenate((image_batch, generated_images))
          y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
          d_loss = d.train_on_batch(X, y)
          print("batch %d d_loss : %f" % (index, d_loss))
          noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
          d.trainable = False
          g_loss = d_on_g.train_on_batch(noise, [1] * BATCH_SIZE)
          d.trainable = True
          print("batch %d g_loss : %f" % (index, g_loss))
          if index % 10 == 9:
              g.save_weights('generator', True)
              d.save_weights('discriminator', True)

In [0]:
def generate(BATCH_SIZE, nice=False):
    g = generator_model()
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    g.load_weights('generator')
    if nice:
      d = discriminator_model()
      d.compile(loss='binary_crossentropy', optimizer="SGD")
      d.load_weight('discriminator')
      noise = np.random.uniform(-1, 1, (BATCH_SIZE*20, 100))
      generated_images = g.predict(noise, verbose=1)
      d_pret = d.predict(generated_images, verbose=1)
      index = np.arrange(0, BATCH_SIZE*20)
      index.resize((BATCH_SIZE*20, 1))
      pre_with_index = list(np.append(d_pret, index, axis=1))
      pre_with_index.sort(key=lambda x: x[0], reverse=True)
      nice_images = np.zeros((BATCH_SIZE,) + generated_images.shape[1:3], dtype=np.float32)
      nice_images = nice_images[:, :, :, None]
      for i in range(BATCH_SIZE):
          idx = int(pre_with_index[i][1])
          nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]
      image = combine_images(nice_images)
    else:
      noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
      generated_images = g.predict(noise, verbose=1)
      image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(
        "generated_image.png")

In [0]:
train(128)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch is 0
Number of baches 468
batch 0 d_loss : 0.707738
batch 0 g_loss : 0.734556
batch 1 d_loss : 0.608143
batch 1 g_loss : 0.687521
batch 2 d_loss : 0.521049
batch 2 g_loss : 0.644967
batch 3 d_loss : 0.472944
batch 3 g_loss : 0.609900
batch 4 d_loss : 0.458554
batch 4 g_loss : 0.581254
batch 5 d_loss : 0.478251
batch 5 g_loss : 0.556706
batch 6 d_loss : 0.512330
batch 6 g_loss : 0.547063
batch 7 d_loss : 0.554953
batch 7 g_loss : 0.531453
batch 8 d_loss : 0.592955
batch 8 g_loss : 0.546069
batch 9 d_loss : 0.622528
batch 9 g_loss : 0.578441
batch 10 d_loss : 0.594730
batch 10 g_loss : 0.644927
batch 11 d_loss : 0.545085
batch 11 g_loss : 0.708752
batch 12 d_loss : 0.474830
batch 12 g_loss : 0.795198
batch 13 d_loss : 0.418485
batch 13 g_loss : 0.872662
batch 14 d_loss : 0.365304
batch 14 g_loss : 0.974714
batch 15 d_loss : 0.291621
batch 15 g_loss : 1.056967
batch 16 d_loss : 0.265367
batch 16 g_loss : 1.096237
batch 17 d_loss : 0.231277
batch 17 g_loss : 1.123003
batch 18 d_loss 


batch 90 d_loss : 0.090748
batch 90 g_loss : 0.008246
batch 91 d_loss : 0.061540
batch 91 g_loss : 0.007616
batch 92 d_loss : 0.071760
batch 92 g_loss : 0.028638
batch 93 d_loss : 0.064672
batch 93 g_loss : 0.032348
batch 94 d_loss : 0.171117
batch 94 g_loss : 0.009110
batch 95 d_loss : 0.110584
batch 95 g_loss : 0.011101
batch 96 d_loss : 0.146137
batch 96 g_loss : 0.020336
batch 97 d_loss : 0.081718
batch 97 g_loss : 0.013123
batch 98 d_loss : 0.102963
batch 98 g_loss : 0.005945
batch 99 d_loss : 0.100006
batch 99 g_loss : 0.019386
batch 100 d_loss : 0.165828
batch 100 g_loss : 0.041011
batch 101 d_loss : 0.065298
batch 101 g_loss : 0.015550
batch 102 d_loss : 0.117279
batch 102 g_loss : 0.043362
batch 103 d_loss : 0.161861
batch 103 g_loss : 0.009280
batch 104 d_loss : 0.122908
batch 104 g_loss : 0.023113
batch 105 d_loss : 0.079261
batch 105 g_loss : 0.025814
batch 106 d_loss : 0.120107
batch 106 g_loss : 0.006109
batch 107 d_loss : 0.199195
batch 107 g_loss : 0.041896
batch 108 d

batch 179 g_loss : 0.039483
batch 180 d_loss : 0.210951
batch 180 g_loss : 0.022058
batch 181 d_loss : 0.206671
batch 181 g_loss : 0.023750
batch 182 d_loss : 0.194101
batch 182 g_loss : 0.036167
batch 183 d_loss : 0.222191
batch 183 g_loss : 0.025092
batch 184 d_loss : 0.249194
batch 184 g_loss : 0.018126
batch 185 d_loss : 0.259050
batch 185 g_loss : 0.037746
batch 186 d_loss : 0.235970
batch 186 g_loss : 0.010925
batch 187 d_loss : 0.309268
batch 187 g_loss : 0.111451
batch 188 d_loss : 0.445625
batch 188 g_loss : 0.004984
batch 189 d_loss : 0.562800
batch 189 g_loss : 0.222785
batch 190 d_loss : 0.764970
batch 190 g_loss : 0.008182
batch 191 d_loss : 0.432455
batch 191 g_loss : 0.085507
batch 192 d_loss : 0.322247
batch 192 g_loss : 0.016752
batch 193 d_loss : 0.254324
batch 193 g_loss : 0.046131
batch 194 d_loss : 0.287825
batch 194 g_loss : 0.010289
batch 195 d_loss : 0.304017
batch 195 g_loss : 0.032083
batch 196 d_loss : 0.239687
batch 196 g_loss : 0.055412
batch 197 d_loss : 0

batch 269 g_loss : 1.185440
batch 270 d_loss : 0.339733
batch 270 g_loss : 0.924697
batch 271 d_loss : 0.367522
batch 271 g_loss : 1.063676
batch 272 d_loss : 0.336237
batch 272 g_loss : 1.063204
batch 273 d_loss : 0.385643
batch 273 g_loss : 1.583164
batch 274 d_loss : 0.352445
batch 274 g_loss : 0.938429
batch 275 d_loss : 0.313388
batch 275 g_loss : 1.457131
batch 276 d_loss : 0.381307
batch 276 g_loss : 1.003666
batch 277 d_loss : 0.416235
batch 277 g_loss : 1.843508
batch 278 d_loss : 0.420169
batch 278 g_loss : 0.829651
batch 279 d_loss : 0.402447
batch 279 g_loss : 2.170353
batch 280 d_loss : 0.507199
batch 280 g_loss : 0.886939
batch 281 d_loss : 0.562129
batch 281 g_loss : 2.619766
batch 282 d_loss : 0.805500
batch 282 g_loss : 0.585482
batch 283 d_loss : 0.679060
batch 283 g_loss : 1.797307
batch 284 d_loss : 0.529944
batch 284 g_loss : 0.742846
batch 285 d_loss : 0.514414
batch 285 g_loss : 1.613822
batch 286 d_loss : 0.526257
batch 286 g_loss : 0.748978
batch 287 d_loss : 0

...

batch 166 g_loss : 1.865280
batch 167 d_loss : 0.325937
batch 167 g_loss : 1.720820
batch 168 d_loss : 0.276743
batch 168 g_loss : 2.280308
batch 169 d_loss : 0.209566
batch 169 g_loss : 2.359123
batch 170 d_loss : 0.557127
batch 170 g_loss : 0.863600
batch 171 d_loss : 0.622902
batch 171 g_loss : 3.885833
batch 172 d_loss : 0.512068
batch 172 g_loss : 1.352414
batch 173 d_loss : 0.360664
batch 173 g_loss : 2.154923
batch 174 d_loss : 0.308496
batch 174 g_loss : 1.351666
batch 175 d_loss : 0.395510
batch 175 g_loss : 1.653470
batch 176 d_loss : 0.355954
batch 176 g_loss : 1.662850
batch 177 d_loss : 0.434176
batch 177 g_loss : 1.877743
batch 178 d_loss : 0.359174
batch 178 g_loss : 1.703159
batch 179 d_loss : 0.305938
batch 179 g_loss : 1.880888
batch 180 d_loss : 0.325228
batch 180 g_loss : 1.675056
batch 181 d_loss : 0.352731
batch 181 g_loss : 1.916462
batch 182 d_loss : 0.380160
batch 182 g_loss : 1.212164
batch 183 d_loss : 0.336901
batch 183 g_loss : 2.076211
batch 184 d_loss : 0

batch 257 g_loss : 2.243638
batch 258 d_loss : 0.460774
batch 258 g_loss : 1.333509
batch 259 d_loss : 0.370065
batch 259 g_loss : 2.655247
batch 260 d_loss : 0.306886
batch 260 g_loss : 1.638775
batch 261 d_loss : 0.246991
batch 261 g_loss : 2.536989
batch 262 d_loss : 0.252958
batch 262 g_loss : 1.860022
batch 263 d_loss : 0.412794
batch 263 g_loss : 1.355889
batch 264 d_loss : 0.361003
batch 264 g_loss : 1.403804
batch 265 d_loss : 0.268377
batch 265 g_loss : 2.724736
batch 266 d_loss : 0.328829
batch 266 g_loss : 1.770063
batch 267 d_loss : 0.266617
batch 267 g_loss : 2.396216
batch 268 d_loss : 0.315823
batch 268 g_loss : 1.373183
batch 269 d_loss : 0.363694
batch 269 g_loss : 3.270492
batch 270 d_loss : 0.567181
batch 270 g_loss : 0.787265
batch 271 d_loss : 0.561754
batch 271 g_loss : 3.407939
batch 272 d_loss : 0.629710
batch 272 g_loss : 0.910640
batch 273 d_loss : 0.666739
batch 273 g_loss : 4.176897
batch 274 d_loss : 0.632652
batch 274 g_loss : 1.092531
batch 275 d_loss : 0

batch 345 d_loss : 0.257247
batch 345 g_loss : 1.717165
batch 346 d_loss : 0.291840
batch 346 g_loss : 2.329989
batch 347 d_loss : 0.297073
batch 347 g_loss : 2.036656
batch 348 d_loss : 0.334209
batch 348 g_loss : 1.724611
batch 349 d_loss : 0.380172
batch 349 g_loss : 2.970026
batch 350 d_loss : 0.432703
batch 350 g_loss : 1.043858
batch 351 d_loss : 0.593525
batch 351 g_loss : 3.394501
batch 352 d_loss : 0.595012
batch 352 g_loss : 0.907261
batch 353 d_loss : 0.557891
batch 353 g_loss : 3.541143
batch 354 d_loss : 0.436863
batch 354 g_loss : 1.622293
batch 355 d_loss : 0.265007
batch 355 g_loss : 1.562299
batch 356 d_loss : 0.411669
batch 356 g_loss : 2.060158
batch 357 d_loss : 0.390877
batch 357 g_loss : 1.214783
batch 358 d_loss : 0.400753
batch 358 g_loss : 2.271102
batch 359 d_loss : 0.406509
batch 359 g_loss : 1.186848
batch 360 d_loss : 0.363989
batch 360 g_loss : 2.443417
batch 361 d_loss : 0.322790
batch 361 g_loss : 1.747516
batch 362 d_loss : 0.280280
batch 362 g_loss : 1

batch 435 d_loss : 0.489574
batch 435 g_loss : 2.009797
batch 436 d_loss : 0.377709
batch 436 g_loss : 1.473215
batch 437 d_loss : 0.340942
batch 437 g_loss : 2.539771
batch 438 d_loss : 0.393107
batch 438 g_loss : 1.158635
batch 439 d_loss : 0.348672
batch 439 g_loss : 2.373971
batch 440 d_loss : 0.332859
batch 440 g_loss : 1.666497
batch 441 d_loss : 0.346411
batch 441 g_loss : 2.355923
batch 442 d_loss : 0.235982
batch 442 g_loss : 2.032634
batch 443 d_loss : 0.349681
batch 443 g_loss : 2.492162
batch 444 d_loss : 0.321042
batch 444 g_loss : 2.578157
batch 445 d_loss : 0.331593
batch 445 g_loss : 2.419189
batch 446 d_loss : 0.291821
batch 446 g_loss : 2.132600
batch 447 d_loss : 0.348911
batch 447 g_loss : 1.796311
batch 448 d_loss : 0.378505
batch 448 g_loss : 2.217254
batch 449 d_loss : 0.331488
batch 449 g_loss : 1.712397
batch 450 d_loss : 0.285076
batch 450 g_loss : 2.280899
batch 451 d_loss : 0.277392
batch 451 g_loss : 2.217638
batch 452 d_loss : 0.277738
batch 452 g_loss : 2

batch 54 d_loss : 0.446449
batch 54 g_loss : 1.480442
batch 55 d_loss : 0.351979
batch 55 g_loss : 2.295902
batch 56 d_loss : 0.508284
batch 56 g_loss : 1.273299
batch 57 d_loss : 0.553454
batch 57 g_loss : 2.756109
batch 58 d_loss : 0.310132
batch 58 g_loss : 2.071898
batch 59 d_loss : 0.293934
batch 59 g_loss : 2.584624
batch 60 d_loss : 0.298363
batch 60 g_loss : 1.661280
batch 61 d_loss : 0.466411
batch 61 g_loss : 1.653807
batch 62 d_loss : 0.552507
batch 62 g_loss : 1.218363
batch 63 d_loss : 0.373950
batch 63 g_loss : 2.033818
batch 64 d_loss : 0.350037
batch 64 g_loss : 1.926893
batch 65 d_loss : 0.372777
batch 65 g_loss : 1.363293
batch 66 d_loss : 0.693776
batch 66 g_loss : 1.216698
batch 67 d_loss : 0.556154
batch 67 g_loss : 1.807836
batch 68 d_loss : 0.378921
batch 68 g_loss : 1.701243
batch 69 d_loss : 0.448595
batch 69 g_loss : 1.715282
batch 70 d_loss : 0.376806
batch 70 g_loss : 1.555014
batch 71 d_loss : 0.455669
batch 71 g_loss : 1.282371
batch 72 d_loss : 0.564558
b

In [0]:
generate(128, False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  This is separate from the ipykernel package so we can avoid doing imports until


128/128 [==============================] - 0s 1ms/step
